In [1]:
from PIL import Image
from utils import FeatureExtractor
from pathlib import Path
import numpy as np
from elasticsearch import Elasticsearch
from os import listdir
from os.path import isfile, join
import argparse
import json
import pathlib
from elasticsearch.helpers import bulk

ImportError: cannot import name 'FeatureExtractor' from 'utils' (unknown location)

In [2]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])

In [3]:
es.cluster.health(wait_for_status='yellow', request_timeout=1)

c:\Users\med_g\AppData\Local\Programs\Python\Python39\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'cluster_name': 'elasticsearch',
 'status': 'green',
 'timed_out': False,
 'number_of_nodes': 1,
 'number_of_data_nodes': 1,
 'active_primary_shards': 2,
 'active_shards': 2,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 0,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 100.0}

In [4]:
index = "image_se"

In [5]:
es.options(ignore_status = [400, 404]).indices.delete(index = index)

AttributeError: 'Elasticsearch' object has no attribute 'options'

In [6]:
settings = {
  "settings": {
    "elastiknn": True,
    "number_of_shards": 1,
    "number_of_replicas": 0
  }
}

mapping = {
  "dynamic": False,
  "properties": {
    "feature_vector": {
        "type": "elastiknn_dense_float_vector",
        "elastiknn": {
          "dims": 2209,
          "model": "lsh",
          "similarity": "l2",
        }
    },
    "source_folder":{"type":"text","index":False},
    "image_name":{"type":"text","index":False}
  }
}

created_index = es.indices.create(
    index = index,
    body = settings,
    ignore = 400
)

C:\Users\med_g\AppData\Local\Temp/ipykernel_7684/2870889880.py:28: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  created_index = es.indices.create(
c:\Users\med_g\AppData\Local\Programs\Python\Python39\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [8]:
es.indices.put_mapping(mapping, index)


C:\Users\med_g\AppData\Local\Temp/ipykernel_7684/273460740.py:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.put_mapping(mapping, index)


{'acknowledged': True}

In [5]:
fe = FeatureExtractor()

In [11]:
inputDestination = "C:/Users/med_g/OneDrive/Desktop/SUPCOM/DB Indexing & geospacial applications/elasticsearch/img_engine"
import csv
with open(inputDestination + '/reducedVggOutputs.csv') as featureVector:
    pcaFeatures = list(csv.reader(featureVector, delimiter=','))
len(pcaFeatures)

9146

In [12]:
len(pcaFeatures[0])

2209

In [15]:
import pickle as pk
imgPaths = pk.load(open(inputDestination + "/imgPaths.pkl",'rb'))


In [19]:
imgPaths

['C:/Users/med_g/OneDrive/Desktop/SUPCOM/DB Indexing & geospacial applications/elasticsearch/bdimage/idb/Objects/accordion/image_0001.jpg',
 'C:/Users/med_g/OneDrive/Desktop/SUPCOM/DB Indexing & geospacial applications/elasticsearch/bdimage/idb/Objects/accordion/image_0002.jpg',
 'C:/Users/med_g/OneDrive/Desktop/SUPCOM/DB Indexing & geospacial applications/elasticsearch/bdimage/idb/Objects/accordion/image_0003.jpg',
 'C:/Users/med_g/OneDrive/Desktop/SUPCOM/DB Indexing & geospacial applications/elasticsearch/bdimage/idb/Objects/accordion/image_0004.jpg',
 'C:/Users/med_g/OneDrive/Desktop/SUPCOM/DB Indexing & geospacial applications/elasticsearch/bdimage/idb/Objects/accordion/image_0005.jpg',
 'C:/Users/med_g/OneDrive/Desktop/SUPCOM/DB Indexing & geospacial applications/elasticsearch/bdimage/idb/Objects/accordion/image_0006.jpg',
 'C:/Users/med_g/OneDrive/Desktop/SUPCOM/DB Indexing & geospacial applications/elasticsearch/bdimage/idb/Objects/accordion/image_0007.jpg',
 'C:/Users/med_g/One

In [23]:
splitPaths = [path.split('/') for path in imgPaths]

In [24]:

parameters = [(path[-2], path[-1]) for path in splitPaths]

In [26]:
parameters[-1]

('yin_yang', 'image_0060.jpg')

In [27]:
len(pcaFeatures) == len(parameters)

True

In [28]:
from contextlib import contextmanager
import sys, os

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

In [32]:
done = 0
DATA_SIZE = len(pcaFeatures)
for i in range(DATA_SIZE):
    done += 1
    imgPcaFeatures = [float(x) for x in pcaFeatures[i]]
    _body = {"source_folder": parameters[i][0], "image_name": parameters[i][1], "feature_vector": imgPcaFeatures}
    with suppress_stdout():
        response = es.index(index=index, id=done, body=_body, request_timeout=60)
    print("[{}/{}] => {:.2f}% Done!                    ".format(done, DATA_SIZE, done/DATA_SIZE), end='\r')

C:\Users\med_g\AppData\Local\Temp/ipykernel_7684/306439095.py:8: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  response = es.index(index=index, id=done, body=_body, request_timeout=60)
c:\Users\med_g\AppData\Local\Programs\Python\Python39\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
